In [25]:
import hashlib
import os
import json
import requests
import sqlalchemy
import pymysql
import pandas as pd
import gzip
import pickle
import joblib
import PyPDF2
import docx
import io

In [26]:
connstr = ""
engine = sqlalchemy.create_engine(connstr)
# toefl = pd.read_sql("select id, convert(content using utf8) content from toefl", engine)

In [45]:
file_hash = {}
dupes = {}
empty = {}
notfound = {}
directory = "../data/Buffer2/"
for filename in os.listdir(directory):
    with open(directory+filename, "r+",encoding='utf-8') as f:
        data = f.read().encode("utf-8")
        key = hashlib.md5(data).hexdigest()
        clean_name = filename.replace(".json","")
        if key in file_hash:
            j = json.loads(data)
            matches = j["matches"]
            if len(matches) > 0:
                orig = toefl[toefl["content"].str.match(matches[0]["sentence"])]
                if orig.shape[0] > 0:
                    dupes[clean_name] = orig["id"].values[0]
                else:
                    notfound[key] = clean_name
            else:
                empty[key] = clean_name
        else:
            file_hash[key] = clean_name

In [46]:
len(dupes)

0

In [47]:
len(empty)

0

In [48]:
len(notfound)

0

In [44]:
API_ENDPOINT = "http://localhost:8081/v2/check"
for index, row in toefl.iterrows():
    id = str(row["id"]) + ".json"
    data = { "language": "en-US", "text": row["content"] }
    r = requests.post(url = API_ENDPOINT, data = data)
    with open('../data/Buffer2/' + id, 'w+') as outfile:  
        json.dump(r.text, outfile)
    if index % 1000 == 0:
        print(index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [80]:
db = engine.connect()
count = dict()
for fn in os.listdir('../data/Buffer2'):
    file_id = fn.strip('.json')
    with open('../data/Buffer2/' + fn, 'r') as f:
        data = json.loads(f.read())
        vals = json.loads(data)
        count[file_id] = len(vals['matches'])
        for m in vals['matches']:
            message = m['message']
            short_message = m['shortMessage']
            offset = m['offset']
            length = m['length']
            context_text = m['context']['text']
            context_offset = m['context']['offset']
            context_length = m['context']['length']
            sentence = m['sentence']
            rule_name = m['rule']['id']
            rule_description = m['rule']['description']
            rule_issue = m['rule']['issueType']
            essay_set = 1
            category_name = m['rule']['category']['id']

            fetch = db.execute('SELECT id from error_rule where text_id = "' + rule_name + '";')
            res = fetch.fetchone()
            if not res:
                fetch = db.execute('SELECT id from error_category where text_id = "' + category_name + '";')
                res = fetch.fetchone()
                cat_id = res[0]
                db.execute('INSERT INTO error_rule(text_id, description, issue_type, category_id) VALUES(%s, %s, %s, %s);', (rule_name, rule_description, rule_issue, cat_id))
                fetch = db.execute("select last_insert_id();")
                rule_id = fetch.fetchone()[0]
            else:
                rule_id = res[0]
            db.execute('INSERT INTO error_matches(message, short_message, offset, length, context_text, context_offset, context_length, sentence, rule_id, file_id, essay_set) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s);', (message, short_message, offset, length, context_text, context_offset, context_length, sentence, rule_id, file_id, essay_set))
            
for k in count.keys():
    fetch = db.execute('select count(id) from error_matches where file_id = ' + k + ' and essay_set = 1;')
    v = fetch.fetchone()
    if v[0] != count[k]:
        print(k, v[0], count[k])

In [3]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [15]:
count_vect = None
tfidf = None
with open("../data/vectorizer.pkl","rb") as f:
    count_vect = pickle.load(f)
with open("../data/tfidf.pkl","rb") as f:
    tfidf = pickle.load(f)

TypeError: file must have 'read' and 'readline' attributes

In [16]:
# creating an object 
file = open('alphazero_preprint.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

text = []
# print the number of pages in pdf file
for i in range(fileReader.numPages):
    p = fileReader.getPage(i)
    text.append(p.extractText())
print(text)

['Ageneralreinforcementlearningalgorithmthat\nmasterschess,shogiandGothroughself-play\nDavidSilver,\n1\n;\n2\n\nThomasHubert,\n1\n\nJulianSchrittwieser,\n1\n\nIoannisAntonoglou,\n1\n;\n2\nMatthewLai,\n1\nArthurGuez,\n1\nMarcLanctot,\n1\nLaurentSifre,\n1\nDharshanKumaran,\n1\n;\n2\nThoreGraepel,\n1\n;\n2\nTimothyLillicrap,\n1\nKarenSimonyan,\n1\nDemisHassabis\n1\n1\nDeepMind,6PancrasSquare,LondonN1C4AG.\n2\nUniversityCollegeLondon,GowerStreet,LondonWC1E6BT.\n\nTheseauthorscontributedequallytothiswork.\nAbstract\nThegameofchessisthelongest-studieddomaininthehistoryofintelligence.\nThestrongestprogramsarebasedonacombinationofsophisticatedsearchtechniques,\nadaptations,andhandcraftedevaluationfunctionsthathavebeen\nbyhumanexpertsoverseveraldecades.Bycontrast,theAlphaGoZeroprogramrecently\nachievedsuperhumanperformanceinthegameofGobyreinforcementlearningfromself-\nplay.Inthispaper,wegeneralizethisapproachintoasingleAlphaZeroalgorithmthatcan\nachievesuperhumanperformanceinmanychallenginggame

In [24]:
with open('Cover Letter.docx', 'rb+') as f:
    bf = io.BytesIO(f.read())
    doc = docx.Document(bf)
    text = []
    # print the number of pages in pdf file
    for p in doc.paragraphs:
        text.append(p.text)
    print(" ".join(text).encode("utf8"))

b'March 22nd, 2017  RE: Software Developer Job Posting   To Dr. Christopher Collins,   I am writing to you to express my great interest in becoming a part of your team as a software developer position for the Open Social Sciences and Humanities Project. Web is a passion of mine, and having thoroughly enjoyed your Human Computer Interaction course in 2013, I believe this opportunity to work together would be of mutual value.  For the past two years, I have been working with a small team of developers on both frontend and backend development of an ecosystem of web-based applications ranging in use for mobile \xe2\x80\x9con-the-job\xe2\x80\x9d users, custom data analytics reporting, and full enterprise solutions. I believe I would be an effective asset for this project.  I take as much initiative as possible towards personal and professional development, and I try to focus this knowledge into avenues I see relevant. I encourage you to check out my online portfolio which can be found at  i

In [30]:
words = pd.read_sql("select * from word limit 100", engine)

In [ ]:
tfidf = pd.read_sql("select * from tfidf", engine)